# Trabalho prático 4: representação de dados em banco de dados relacional


## Criando a estrutura do banco


In [3]:
import sqlite3
import pandas as pd
caminho = './'
banco = 'banco.db'

conexao = sqlite3.connect(caminho + banco)

script_cria_tabela_indicadores = "\
    CREATE TABLE  IF NOT EXISTS indicadores (\
        indicador_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\
        codigo text NOT NULL,\
        nome text NOT NULL,\
        UNIQUE(codigo)\
    );"
conexao.executescript(script_cria_tabela_indicadores)

script_cria_tabela_dados = "\
    CREATE TABLE IF NOT EXISTS paises (\
        pais_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,\
        nome_curto text NOT NULL,\
        nome_longo text NOT NULL,\
        regiao text NOT NULL,\
        UNIQUE(nome_curto)\
    );"
conexao.executescript(script_cria_tabela_dados)

script_cria_tabela_dados  = "\
    CREATE TABLE IF NOT EXISTS dados ( \
        dado_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, \
        ano INTEGER NOT NULL,\
        valor REAL NOT NULL,\
        pais_id INTEGER NOT NULL,\
        indicador_id INTEGER NOT NULL,\
        FOREIGN KEY (pais_id) REFERENCES paises (pais_id) ON DELETE CASCADE ON UPDATE CASCADE,\
        FOREIGN KEY (indicador_id) REFERENCES indicadores (indicador_id) ON DELETE CASCADE ON UPDATE CASCADE\
    );"
conexao.executescript(script_cria_tabela_dados)

tabelas = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table'", conexao)

conexao.commit()
conexao.close()

display(tabelas)


,name
0,indicadores
1,sqlite_sequence
2,paises
3,dados


## Inserindo dados na tabela países


In [1]:
import pandas as pd
import sqlite3

caminho: str = './'
arquivo: str = 'WDICountry.csv'
banco: str = 'banco.db'

conexao = sqlite3.connect(caminho + banco)

df_paises = pd.read_csv(caminho + arquivo, encoding='utf_8',
                        encoding_errors='surrogateescape')


def codigo_pais_existe(code: str) -> bool:
    cursor = conexao.cursor()
    # conta a quantidade do codigo na tabela
    query = f"SELECT count(*) FROM paises WHERE nome_curto LIKE \'{code}\'"
    resultado = cursor.execute(query)
    # se tiver algum retorna verdadeiro
    return resultado.fetchone()[0] >= 1


for coluna, linha in df_paises.iterrows():
    code = linha['Country Code']
    # só insere se não existe aquele código
    if not(codigo_pais_existe(code)):
        name = linha['Long Name']
        region = linha['Region']
        script = f"INSERT INTO paises (nome_curto, nome_longo, regiao) VALUES (\"{code}\",\"{name}\",\"{region}\");"
        conexao.executescript(script)

tabela_paises = pd.read_sql_query("SELECT * FROM paises", conexao)

conexao.close()

display(tabela_paises)


,pais_id,nome_curto,nome_longo,regiao
0,1,ABW,Aruba,Latin America & Caribbean
1,2,AFG,Islamic State of Afghanistan,South Asia
2,3,AGO,People's Republic of Angola,Sub-Saharan Africa
3,4,ALB,Republic of Albania,Europe & Central Asia
4,5,AND,Principality of Andorra,Europe & Central Asia
...,...,...,...,...
258,259,XKX,Republic of Kosovo,Europe & Central Asia
259,260,YEM,Republic of Yemen,Middle East & North Africa
260,261,ZAF,Republic of South Africa,Sub-Saharan Africa
261,262,ZMB,Republic of Zambia,Sub-Saharan Africa


## Inserindo dados na tabela indicadores

In [4]:
import pandas as pd
import sqlite3

caminho: str = './'
arquivo: str = 'WDIData.csv'
banco: str = 'banco.db'
conexao = sqlite3.connect(caminho + banco)

df_indicadores = pd.read_csv(
    caminho + arquivo, encoding='utf_8', encoding_errors='surrogateescape')


df_indicadores = df_indicadores[[
    'Indicator Name', 'Indicator Code']].drop_duplicates(subset=["Indicator Code"])


def codigo_indicador_existe(code: str) -> bool:
    cursor = conexao.cursor()
    # conta a quantidade do codigo indicador na tabela
    query = f"SELECT count(*) FROM indicadores WHERE codigo LIKE \'{code}\'"
    resultado = cursor.execute(query)
    # se tiver algum retorna verdadeiro
    return resultado.fetchone()[0] >= 1


for coluna, linha in df_indicadores.iterrows():
    code = linha['Indicator Code']
    if(not codigo_indicador_existe(code)):
        name = linha['Indicator Name']
        script = f"INSERT INTO indicadores (codigo, nome) VALUES (\"{code}\",\"{name}\");"
        conexao.executescript(script)

tabela_indicadores = pd.read_sql_query("SELECT * FROM indicadores", conexao)
conexao.close()

display(tabela_indicadores)


,indicador_id,codigo,nome
0,1,PA.NUS.PPP.05,"2005 PPP conversion factor, GDP (LCU per inter..."
1,2,PA.NUS.PRVT.PP.05,"2005 PPP conversion factor, private consumptio..."
2,3,EG.CFT.ACCS.ZS,Access to clean fuels and technologies for coo...
3,4,EG.ELC.ACCS.ZS,Access to electricity (% of population)
4,5,EG.ELC.ACCS.RU.ZS,"Access to electricity, rural (% of rural popul..."
...,...,...,...
1594,1595,SG.VAW.NEGL.ZS,Women who believe a husband is justified in be...
1595,1596,SG.VAW.REFU.ZS,Women who believe a husband is justified in be...
1596,1597,SP.M15.2024.FE.ZS,Women who were first married by age 15 (% of w...
1597,1598,SP.M18.2024.FE.ZS,Women who were first married by age 18 (% of w...


## Inserindo dados na tabela dados

In [11]:
import pandas as pd
import sqlite3

caminho: str = './'
arquivo: str = 'WDIData.csv'
banco: str = 'banco.db'
conexao = sqlite3.connect(caminho + banco)

df_dados = pd.read_csv(
    caminho + arquivo, encoding='utf_8', encoding_errors='surrogateescape')


def obter_indicador_id(code: str) -> int:
    cursor = conexao.cursor()
    query = f"SELECT indicador_id FROM indicadores WHERE codigo LIKE \'{code}\'"
    resultado = cursor.execute(query)
    return resultado.fetchone()[0]


def obter_pais_id(code: str) -> int:
    cursor = conexao.cursor()
    query = f"SELECT pais_id FROM paises WHERE nome_curto LIKE \'{code}\'"
    resultado = cursor.execute(query)
    return resultado.fetchone()[0]


# necessario para limitar a insercao dos dados devido ao tamanho do dataset
limite = 30000
for coluna, linha in df_dados.iterrows():
    if(limite >= 0):
        indicador_id = obter_indicador_id(linha['Indicator Code'])
        pais_id = obter_pais_id(linha['Country Code'])
        for ano in range(1960, 2018, 1):
            cursor = conexao.cursor()
            valor = linha[str(ano)]
            script = f"INSERT INTO dados (ano, valor, pais_id, indicador_id) VALUES (\"{ano}\",\"{valor}\", \"{pais_id}\", \"{indicador_id}\");"
            cursor.execute(script)
    limite -= 1

tabela_dados = pd.read_sql_query(
    "SELECT d.dado_id, d.ano, d.valor, i.nome, p.nome_longo  FROM dados d inner join paises p on d.pais_id = p.pais_id inner join indicadores i on i.indicador_id = d.indicador_id", conexao)
conexao.close()

display(tabela_dados)


KeyboardInterrupt: 